In [1]:
from gurobipy import *  # import the optimize solver Gurobi

In [2]:
number_of_month = 4 # Set the index for number of month(1, 2, 3, 4)
m = Model() # Import and create the model

Using license file C:\Users\chaoz\gurobi_licence\gurobi.lic
Academic license - for non-commercial use only


In [3]:
# Set the input Parameter: 
unit_production_cost = [50, 80, 40, 70]  # Cost of production per unit in each month  
unit_storage_cost = 20  # Storage cost incurred per unit  
demand = [500, 650, 1000, 700]   # Demand in each month 
unit_selling_price = 60  # Selling price per unit for left over units at the end of month 4
initial_inventory = 0  # Inventory at the beginning of month 1

In [4]:
# Set the Variable list: Number of units produced in each month
# Set the variable nx to integer number
nx = [] 
for i in range(number_of_month):
    nx.append(m.addVar(vtype=GRB.CONTINUOUS, name='nx{}'.format(i + 1)))

# Caculate inventory at the end of each month
ni = []
for i in range(len(nx)):
    if i > 0:
        initial_inventory = ni[i-1]
    xx = nx[i] + initial_inventory - demand[i]
    ni.append(xx)

In [5]:
# Set the Minimize Obijective: Net Cost
m.setObjective(quicksum([unit_production_cost[i]*nx[i] for i in range(len(nx))]) + quicksum([unit_storage_cost*ni[i] for i in range(len(nx))]) - ni[3]*unit_selling_price,  GRB.MINIMIZE)

In [6]:
# Set Non Negative decision variable
c1 = []
for i in range(len(nx)):
    c1.append(m.addConstr(nx[i] >= 0))
    
# End of month inventory ni is non negative 
c2 = []
for i in range(len(nx)):
    c2.append(m.addConstr(ni[i] >= 0))

In [7]:
# Run the optimize solver
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 8 rows, 4 columns and 14 nonzeros
Model fingerprint: 0x372b0f0e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 3e+03]
Presolve removed 8 rows and 4 columns
Presolve time: 0.02s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5250000e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.04 seconds
Optimal objective  1.525000000e+05


In [8]:
# Get the Optimal Solution for X
m.printAttr('X')


    Variable            X 
-------------------------
         nx1         1150 
         nx3         1700 


In [9]:
# Get the Optimal Objective Value
m.ObjVal

152500.0